In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np

In [2]:
# req_url = 'https://www.sejm.gov.pl/Sejm8.nsf/agent.xsp?symbol=posglos&NrKadencji=8'
req_url = 'https://www.sejm.gov.pl/Sejm8.nsf/agent.xsp'
req_params = dict(symbol='posglos', NrKadencji=8)
source_url = 'https://www.sejm.gov.pl/Sejm8.nsf/'

In [3]:
try:
    r = requests.get(req_url, req_params)
except Exception as e:
    print(e)
else:
    print(f"Status code:  {r.status_code}")
    print(f"Content type: {r.headers['content-type']}")
    # print(r.text)
    html = r.text

Status code:  200
Content type: text/html;charset=utf-8


In [4]:
#soup = BeautifulSoup(html, 'html.parser')
soup = BeautifulSoup(html, 'lxml')

In [5]:
df = pd.DataFrame()

In [6]:
l = []

In [7]:
for a in soup.table.find_all('a'):
    l.append([a.text,source_url+a.get('href')])
    #print(a.text,'\t',a.get('href'))

In [8]:
df1 = pd.DataFrame(l,columns=['Data pos. Sejmu','Link'])

In [9]:
voting_df = pd.read_html('https://www.sejm.gov.pl/Sejm8.nsf/agent.xsp?symbol=posglos&NrKadencji=8')[0]

In [10]:
voting_df.fillna(method='ffill', inplace=True)

In [11]:
voting_df = voting_df.iloc[:,:3]

In [12]:
voting_df.iloc[:,1] = df1.iloc[:,0]

In [13]:
voting_df['Link'] = df1.iloc[:,1]

voting_df

In [14]:
def date_parser(date_str):
    
    month_dict = {'stycznia' : 1,
                  'lutego' : 2,
                  'marca' : 3,
                  'kwietnia' : 4,
                  'maja' : 5,
                  'czerwca' : 6,
                  'lipca' : 7,
                  'sierpnia': 8,
                  'września' : 9,
                  'października': 10,
                  'listopada' : 11,
                  'grudnia': 12                 
                 }
    day, month, year, *_ = date_str.split()
    return pd.datetime(int(year), int(month_dict[month]), int(day))

In [15]:
voting_df['Data'] = voting_df.iloc[:,1].apply(date_parser)

In [16]:
voting_df

,Nr pos. Sejmu,Data pos. Sejmu,Liczba gÅosowaÅ,Link,Data
0,86.0,11 września 2019 r.,88,https://www.sejm.gov.pl/Sejm8.nsf/agent.xsp?sy...,2019-09-11
1,86.0,16 października 2019 r.,38,https://www.sejm.gov.pl/Sejm8.nsf/agent.xsp?sy...,2019-10-16
2,85.0,31 lipca 2019 r.,24,https://www.sejm.gov.pl/Sejm8.nsf/agent.xsp?sy...,2019-07-31
3,85.0,9 sierpnia 2019 r.,11,https://www.sejm.gov.pl/Sejm8.nsf/agent.xsp?sy...,2019-08-09
4,85.0,30 sierpnia 2019 r.,38,https://www.sejm.gov.pl/Sejm8.nsf/agent.xsp?sy...,2019-08-30
5,84.0,17 lipca 2019 r.,7,https://www.sejm.gov.pl/Sejm8.nsf/agent.xsp?sy...,2019-07-17
6,84.0,19 lipca 2019 r.,169,https://www.sejm.gov.pl/Sejm8.nsf/agent.xsp?sy...,2019-07-19
7,83.0,3 lipca 2019 r.,7,https://www.sejm.gov.pl/Sejm8.nsf/agent.xsp?sy...,2019-07-03
8,83.0,4 lipca 2019 r.,111,https://www.sejm.gov.pl/Sejm8.nsf/agent.xsp?sy...,2019-07-04
9,82.0,12 czerwca 2019 r.,6,https://www.sejm.gov.pl/Sejm8.nsf/agent.xsp?sy...,2019-06-12


In [17]:
voting_df.to_csv('voting.csv')